O objetivo deste arquivo é experimentar diferentes parâmetros para rodar os modelos,
tentando obter resultados melhores

In [186]:
import pandas as pd
import numpy as np
import utils
from sktime.split import ExpandingWindowSplitter, temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
import epftools as epf
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError, MeanAbsoluteError, MeanSquaredError
from sktime.forecasting.compose import make_reduction
from sklearn.preprocessing import StandardScaler
# Modelos
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [187]:
# Variáveis globais
FORECASTING_HORIZON = 2 # Deve ser a única coisa a mudar entre todos os arquivos h{i}.ipynb
TEST_SIZE = 79
WINDOW_LENGTH = 13
LINEAR_MODELS = ['ridge', 'lasso'] # Modelos cujos dados precisam ser normalizados
N_JOBS = 4
# Estimador sendo ajustado
CURRENT_ESTIMATOR = 'lasso'
# Parâmetros específicos de modelos
# LASSO
LASSO_MAX_ITER = 2000
LASSO_CONFIG = {
    'max_iter': 2000,
    'n_jobs': N_JOBS
}
# Random Forest
RF_MAX_FEATURES = 1.0
N_TREES = 1000
RF_CRITERION = "squared_error"
# LGBM
LGBM_CONFIG = {
    'n_jobs': N_JOBS,
    'verbose': -1,
    'force_col_wise': True,
    'n_estimators': 1000,
    'learning_rate': 0.01,
}

In [188]:
# Importa previsões originais
og_preds = utils.read_and_change_index(f"previsões/h{FORECASTING_HORIZON}.csv")
# Importa dados de inflação
y_true = utils.read_and_change_index("dados/dados_arima.csv")
_, y_true = temporal_train_test_split(y_true, test_size=79 - FORECASTING_HORIZON + 1)
og_preds['y_true'] = y_true
og_preds.head()

,ridge,lasso,random_forest,lgbm,arima,y_true
month,,,,,,
2018-02,0.711133,0.501378,0.34880,0.485688,0.428037,0.32
2018-03,0.556082,0.523392,0.38276,0.477211,0.440431,0.09
2018-04,0.207642,0.269854,0.20582,0.191435,0.388911,0.22
2018-05,-0.168238,0.222377,0.19698,0.111983,0.422620,0.40
2018-06,-0.126009,0.219213,0.20791,0.177197,0.298915,1.26


In [189]:
# Importa dados para previsões
df = utils.read_and_change_index("dados/dados_transformados.csv")
df.head()

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,...,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,crb_metal
month,,,,,,,,,,,,,,,,,,,,,
2004-01,0.76,0.68,-0.548387,-11559013.0,-22319905.0,-18028683.0,-17053471.0,-384.95,-0.0735,-0.363,...,2481.49,0.4,49.609174,-2.34359,8.9,14.696341,348.7366,7.70,18.70,15.10
2004-02,0.61,0.90,0.000000,1650795.0,11117714.0,1275340.0,-2131936.0,-96.42,0.0785,1.699,...,29.00,0.3,121.394093,1.00000,-0.8,15.073171,0.0000,10.11,14.74,24.27
2004-03,0.47,0.40,-0.112903,-1575433.0,-1282702.0,300227.0,-4199993.0,387.24,-0.0248,-2.777,...,2375.73,0.2,-166.416169,-5.00000,-10.1,16.078049,-10382.5684,4.04,2.71,10.95
2004-04,0.37,0.21,-0.270430,539382.0,-378329.0,-4736024.0,-2778970.0,-2535.03,0.0005,-1.562,...,-1911.68,0.2,21.276658,-4.00000,-5.3,16.454878,-339.4192,-0.45,6.35,-11.13
2004-05,0.51,0.54,-0.116667,1252881.0,13051576.0,26890443.0,11143701.0,-62.56,0.1944,4.213,...,1488.90,0.2,14.892162,17.00000,16.2,15.324390,238.2590,-5.31,-15.02,8.03


In [190]:
# Roda o LASSO
if CURRENT_ESTIMATOR == 'lasso':
    lasso = LassoCV(**LASSO_CONFIG)
    forecaster = make_reduction(lasso, window_length=WINDOW_LENGTH)
    # Normaliza dados
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(df)
    normalized_df = pd.DataFrame(normalized_data, columns=df.columns, index=df.index)
    y_train, y_test, y_pred = utils.evaluate_pipeline(forecaster, normalized_df, steps_ahead=FORECASTING_HORIZON)
    # Desnormaliza previsões
    y_pred = y_pred * np.sqrt(scaler.var_[0]) + scaler.mean_[0]

C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.999e-03, tolerance: 7.656e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.245e-02, tolerance: 1.169e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loca

In [191]:
# Roda o Random Forest
if CURRENT_ESTIMATOR == 'random_forest':
    rf = RandomForestRegressor(
        max_features=RF_MAX_FEATURES,
        n_jobs=N_JOBS,
        n_estimators=N_TREES,
        criterion=RF_CRITERION
    )
    forecaster = make_reduction(rf, window_length=WINDOW_LENGTH)
    y_train, y_test, y_pred = utils.evaluate_pipeline(forecaster, df, steps_ahead=FORECASTING_HORIZON)

In [192]:
# Roda o LGBM
if CURRENT_ESTIMATOR == 'lgbm':
    lgbm = LGBMRegressor(**LGBM_CONFIG)
    forecaster = make_reduction(lgbm, window_length=WINDOW_LENGTH)
    y_train, y_test, y_pred = utils.evaluate_pipeline(forecaster, df, steps_ahead=FORECASTING_HORIZON)

In [193]:
# Calcula métricas de erro
# Evita avisos chatos
og_preds['y_true'].name = y_pred.name
# Resultados
mae = MeanAbsoluteError()
mse = MeanSquaredError()
mape = MeanAbsolutePercentageError()
labels = ['MAE', 'MSE', 'MAPE', 'DM']
res = {
    f'new_{CURRENT_ESTIMATOR}': [
        mae(og_preds['y_true'], y_pred),
        mse(og_preds['y_true'], y_pred),
        mape(og_preds['y_true'], y_pred),
        epf.DM(og_preds['y_true'], og_preds['arima'], y_pred)
    ],
    f'old_{CURRENT_ESTIMATOR}': [
        mae(og_preds['y_true'], og_preds[CURRENT_ESTIMATOR]),
        mse(og_preds['y_true'], og_preds[CURRENT_ESTIMATOR]),
        mape(og_preds['y_true'], og_preds[CURRENT_ESTIMATOR]),
        epf.DM(og_preds['y_true'], og_preds['arima'], og_preds[CURRENT_ESTIMATOR])
    ]
}
res_df = pd.DataFrame(res, index=labels)
res_df

C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\performance_metrics\forecasting\_classes.py:467: UserWarning: y_pred and y_true do not have the same column index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\performance_metrics\forecasting\_classes.py:467: UserWarning: y_pred and y_true do not have the same column index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\performance_metrics\forecasting\_classes.py:467: UserWarning: y_pred and y_true do not have the same column index.

,new_lasso,old_lasso
MAE,0.347551,0.347441
MSE,0.199175,0.200124
MAPE,1.481002,1.473030
DM,0.129911,0.135638
